<a href="https://colab.research.google.com/github/Majumota/ConstruccionDB_12FEB/blob/main/Construcci%C3%B3n_DB_Taller_BI_Majumota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Certainly! Designing a database schema for an online merch store requires understanding the entities involved and their relationships. Here's a basic schema for such a store:


*We'll use SQLite, a lightweight database that's integrated with Python, to create this online merch store database.*

Here's a Python script to set up the described database schema:

In [1]:
import sqlite3

**Creación de base de datos**

In [2]:
def create_carmax_v2_db():
    # Connect to SQLite database (it will create 'carmax_v2.db' if it doesn't exist)
    connection = sqlite3.connect('carmax_v2.db')
    cursor = connection.cursor()

       # Product table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Product (
        idProduct INTEGER PRIMARY KEY,
        nameProduct VARCHAR(255) NOT NULL,
        description TEXT NOT NULL,
        price DECIMAL(10,2) NOT NULL,
        stock INTEGER NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Customer table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Customer (
        idCustomer INTEGER PRIMARY KEY,
        nameCustomer VARCHAR(255) NOT NULL,
        email TEXT NOT NULL,
        phone INTEGER NOT NULL UNIQUE,
        address TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Sales table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Sales (
        idSales INTEGER PRIMARY KEY,
        idCustomer INTEGER NOT NULL,
        idProduct INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        total_price DECIMAL(10,2) NOT NULL,
        sale_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (idCustomer) REFERENCES Customer(idCustomer),
        FOREIGN KEY (idProduct) REFERENCES Product(idProduct)
    )
    ''')

    # Commit changes and close connection
    connection.commit()
    connection.close()

# Execute the function to create the database and tables
create_carmax_v2_db()


In [3]:
print (create_carmax_v2_db)

<function create_carmax_v2_db at 0x7ec384124c20>


An Entity-Relationship (ER) model represents the logical structure of a database. The ER diagram showcases entities, their attributes, and the relationships between them.

Here's an ER diagram based on the database schema for the online merch store:

In [4]:
import sqlite3

def print_database():
    # Connect to the SQLite database
    connection = sqlite3.connect('carmax_v2.db')
    cursor = connection.cursor()

    # List of tables in our schema
    tables = ['Product', 'Customer', 'Sales']

    # Iterate over each table and print its contents
    for table in tables:
        print(f"===== {table} =====")

        # Fetch all rows from the table
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()

        # Fetch column names for the table
        cursor.execute(f"PRAGMA table_info({table})")
        columns = [column[1] for column in cursor.fetchall()]

        # Print column names
        print(", ".join(columns))

        # Print rows
        for row in rows:
            print(row)

        # Add a newline for separation
        print("\n")

    # Close the database connection
    connection.close()

# Execute the function to print the database contents
print_database()


===== Product =====
idProduct, nameProduct, description, price, stock, created_at


===== Customer =====
idCustomer, nameCustomer, email, phone, address, created_at


===== Sales =====
idSales, idCustomer, idProduct, quantity, total_price, sale_date




**Importar datos random con faker**

In [16]:
!pip install faker

**Poblamiento de las tablas Product, Customer, Sale con datos random**

In [17]:
import sqlite3
import random
from faker import Faker

# Inicializar Faker en español para Colombia
fake = Faker("es_CO")

In [19]:
# Lista de productos del sector automotriz
automotive_products = [
    ("Aceite de Motor", "Aceite sintético de alta calidad para motor."),
    ("Filtro de Aire", "Filtro de aire para vehículos livianos y pesados."),
    ("Batería 12V", "Batería de alto rendimiento para autos."),
    ("Pastillas de Freno", "Juego de pastillas de freno delanteras."),
    ("Llantas Radiales", "Llantas de alta resistencia para carretera."),
    ("Anticongelante", "Líquido refrigerante para motores."),
    ("Amortiguadores", "Amortiguadores para suspensión delantera."),
    ("Bujías", "Juego de bujías de iridio para encendido eficiente."),
    ("Aceite de Transmisión", "Lubricante para transmisión automática."),
    ("Kit de Correa de Distribución", "Juego de correa y tensores para motor.")
]

# Conectar a la base de datos
connection = sqlite3.connect('carmax_v2.db')
cursor = connection.cursor()

# Limpiar las tablas antes de poblar (opcional, si deseas regenerar datos)
cursor.execute("DELETE FROM Sales")
cursor.execute("DELETE FROM Customer")
cursor.execute("DELETE FROM Product")
connection.commit()

# Poblar la tabla Product con productos automotrices
products = []
for name, description in automotive_products:
    price = round(random.uniform(20, 500), 2)  # Precio entre 20 y 500
    stock = random.randint(5, 50)  # Stock entre 5 y 50
    cursor.execute("INSERT INTO Product (nameProduct, description, price, stock) VALUES (?, ?, ?, ?)",
                   (name, description, price, stock))
    products.append(cursor.lastrowid)  # Guardar ID del producto insertado

# Poblar la tabla Customer con clientes en Colombia
customers = []
for _ in range(10):
    name = fake.name()
    email = fake.unique.email()
    phone = fake.unique.numerify(text="3#########")  # Simula un número colombiano
    address = fake.street_address() + ", " + fake.city() + ", " + fake.department()
    cursor.execute("INSERT INTO Customer (nameCustomer, email, phone, address) VALUES (?, ?, ?, ?)",
                   (name, email, phone, address))
    customers.append(cursor.lastrowid)  # Guardar ID del cliente insertado

# Poblar la tabla Sales con ventas realistas
for _ in range(15):
    idCustomer = random.choice(customers)
    idProduct = random.choice(products)

    # Obtener stock del producto
    cursor.execute("SELECT stock, price FROM Product WHERE idProduct = ?", (idProduct,))
    stock, price = cursor.fetchone()

    if stock > 0:
        quantity = random.randint(1, min(stock, 5))  # Vender entre 1 y stock disponible
        total_price = round(quantity * price, 2)

        cursor.execute("INSERT INTO Sales (idCustomer, idProduct, quantity, total_price) VALUES (?, ?, ?, ?)",
                       (idCustomer, idProduct, quantity, total_price))

        # Reducir el stock del producto vendido
        cursor.execute("UPDATE Product SET stock = stock - ? WHERE idProduct = ?", (quantity, idProduct))

# Guardar los cambios y cerrar la conexión
connection.commit()
connection.close()

print("Base de datos poblada correctamente con datos automotrices y ubicaciones en Colombia.")

Base de datos poblada correctamente con datos automotrices y ubicaciones en Colombia.


**Imprimir tablas con datos**

In [20]:
!pip install pandas

In [25]:
import sqlite3
import pandas as pd

In [26]:
def print_database():
    # Conectar a la base de datos
    connection = sqlite3.connect('carmax_v2.db')

    # Listado de tablas
    tables = ['Product', 'Customer', 'Sales']

    # Iterar sobre cada tabla y mostrarla con pandas
    for table in tables:
        print(f"\n===== {table} =====\n")

        # Leer los datos de la tabla con pandas
        df = pd.read_sql_query(f"SELECT * FROM {table}", connection)

        # Mostrar la tabla en formato DataFrame
        display(df)

    # Cerrar la conexión
    connection.close()

# Ejecutar la función para imprimir la base de datos
print_database()



===== Product =====



,idProduct,nameProduct,description,price,stock,created_at
0,1,Aceite de Motor,Aceite sintético de alta calidad para motor.,158.48,22,2025-02-13 00:07:02
1,2,Filtro de Aire,Filtro de aire para vehículos livianos y pesados.,64.75,8,2025-02-13 00:07:02
2,3,Batería 12V,Batería de alto rendimiento para autos.,33.33,4,2025-02-13 00:07:02
3,4,Pastillas de Freno,Juego de pastillas de freno delanteras.,330.34,33,2025-02-13 00:07:02
4,5,Llantas Radiales,Llantas de alta resistencia para carretera.,231.58,21,2025-02-13 00:07:02
5,6,Anticongelante,Líquido refrigerante para motores.,29.15,30,2025-02-13 00:07:02
6,7,Amortiguadores,Amortiguadores para suspensión delantera.,426.99,19,2025-02-13 00:07:02
7,8,Bujías,Juego de bujías de iridio para encendido efici...,346.54,36,2025-02-13 00:07:02
8,9,Aceite de Transmisión,Lubricante para transmisión automática.,317.51,14,2025-02-13 00:07:02
9,10,Kit de Correa de Distribución,Juego de correa y tensores para motor.,133.76,38,2025-02-13 00:07:02



===== Customer =====



,idCustomer,nameCustomer,email,phone,address,created_at
0,1,Armando Flórez Ramírez,mosqueracristina@example.org,3975997174,"Avenida Magdalena # 21Z-62 Sur, Lebrija, Cesar",2025-02-13 00:07:02
1,2,Lucila Pineda,carlosgonzalez@example.net,3568559909,"Carrera 127 # 9F-1 Sur, Colosó, Santander",2025-02-13 00:07:02
2,3,Antonio Carlos Ruiz,pbarrios@example.com,3145834351,"Carrera 123 # 98-9, Sativanorte, Antioquia",2025-02-13 00:07:02
3,4,Laura Elena Vanegas,vanessa86@example.com,3644849504,"Calle 25 # 51-29 Este, Gualmatán, Guaviare",2025-02-13 00:07:02
4,5,Carmen Irene Rodríguez,caceresaida@example.net,3350953862,"Calle 2ª # 60-71 Sur, Mocoa, Archipiélago de S...",2025-02-13 00:07:02
5,6,Francisco Ricardo García Giraldo,tsolano@example.com,3658920608,"Avenida Purísima de la Concepción # 9-3, Cocor...",2025-02-13 00:07:02
6,7,Mary Marina Naranjo,thernandez@example.com,3430350555,"Dg. 85 # 61-56, San Juan de Urabá, Boyacá",2025-02-13 00:07:02
7,8,Juan Ocampo Castillo,jessicagutierrez@example.com,3762739291,"Cr. 173 # 2-31, Ubalá, Bolívar",2025-02-13 00:07:02
8,9,Iván Castro Luna,rosa85@example.org,3828342821,"Dg. 190 # 4-3 Sur, Subachoque, Tolima",2025-02-13 00:07:02
9,10,Carlos Ramírez,wsuarez@example.org,3825119432,"Carrera 9ª # 98-9 Este, Fúquene, Amazonas",2025-02-13 00:07:02



===== Sales =====



,idSales,idCustomer,idProduct,quantity,total_price,sale_date
0,1,2,5,2,463.16,2025-02-13 00:07:02
1,2,3,1,1,158.48,2025-02-13 00:07:02
2,3,4,10,5,668.80,2025-02-13 00:07:02
3,4,6,7,5,2134.95,2025-02-13 00:07:02
4,5,5,10,2,267.52,2025-02-13 00:07:02
5,6,7,1,4,633.92,2025-02-13 00:07:02
6,7,10,7,2,853.98,2025-02-13 00:07:02
7,8,6,4,2,660.68,2025-02-13 00:07:02
8,9,6,8,4,1386.16,2025-02-13 00:07:02
9,10,3,5,2,463.16,2025-02-13 00:07:02
